# BioByIA: Pipeline Completo de Inteligência Artificial Médica

Este notebook demonstra o pipeline completo do projeto **BioByIA**, abrangendo desde a preparação de dados e fine-tuning do modelo Llama-3-8B até a implementação de um sistema RAG (Retrieval-Augmented Generation) para respostas médicas fundamentadas.

---

## 1. Setup e Importações

Prepare o ambiente instalando as dependências e importando os módulos necessários.

In [ ]:
# 1.1 INSTALAÇÃO
print("📦 Instalando dependências...")
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -q --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install -q transformers datasets pinecone-client langchain langchain-community langchain-google-genai google-generativeai python-dotenv tenacity tiktoken sentence-transformers tqdm
print("✅ Dependências instaladas!")

In [ ]:
# 1.2 IMPORTAÇÕES
import os
import torch
import json
import re
import google.generativeai as genai
from pathlib import Path
from datasets import load_dataset
from pinecone import Pinecone
from unsloth import FastLanguageModel, is_bfloat16_supported, get_chat_template
from trl import SFTTrainer
from transformers import TrainingArguments
from langchain_google_genai import GoogleGenerativeAIEmbeddings

try:
    from google.colab import userdata
    COLAB_AVAILABLE = True
except ImportError:
    COLAB_AVAILABLE = False

print("✅ Importações concluídas!")

## 2. Configuração de Credenciais

Para que o RAG e o Gemini funcionem, você precisa definir suas chaves de API. No Colab, use o menu **Secrets** (ícone de chave 🔑).

In [ ]:
# 2.1 RECUPERAÇÃO DE KEYS
def get_api_key(name):
    if COLAB_AVAILABLE:
        try:
            return userdata.get(name)
        except:
            return os.environ.get(name, "")
    return os.environ.get(name, "")

PINECONE_KEY = get_api_key('PINECONE_API_KEY')
GEMINI_KEY = get_api_key('GEMINI_API_KEY')

if not PINECONE_KEY or not GEMINI_KEY:
    print("⚠️  ATENÇÃO: Chaves não encontradas! Certifique-se de adicioná-las aos 'Secrets' do Colab com os nomes:")
    print("   - PINECONE_API_KEY")
    print("   - GEMINI_API_KEY")
else:
    os.environ["PINECONE_API_KEY"] = PINECONE_KEY
    os.environ["GEMINI_API_KEY"] = GEMINI_KEY
    print("✅ Chaves configuradas com sucesso!")

## 3. Pipeline de Fine-Tuning

Preparação do modelo BioByIA com LoRA e ChatML.

In [ ]:
# 3.1 SETUP DO MODELO
MAX_SEQ_LENGTH = 2048
MODEL_NAME = "unsloth/llama-3-8b-Instruct-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = MAX_SEQ_LENGTH,
    load_in_4bit = True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)
print("✅ Modelo e LoRA prontos!")

## 4. Pipeline RAG

Configuração de busca vetorial no Pinecone.

In [ ]:
# 4.1 INDEX & EMBEDDINGS
PINECONE_INDEX_NAME = "biobyia"
PINECONE_NAMESPACE = "medical_qa"

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY", ""))
index = pc.Index(PINECONE_INDEX_NAME)

embeddings_gen = GoogleGenerativeAIEmbeddings(
    model="text-embedding-004",
    google_api_key=os.environ.get("GEMINI_API_KEY", "")
)

def query_rag_context(query, top_k=5):
    query_vector = embeddings_gen.embed_query(query)
    response = index.query(vector=query_vector, top_k=top_k, include_metadata=True, namespace=PINECONE_NAMESPACE)
    return [{
        "id": m['metadata'].get('article_id', 'N/A'),
        "source": m['metadata'].get('source', 'N/A'), 
        "text": m['metadata'].get('text', ''),
        "score": m['score']
    } for m in response['matches']]
print("✅ Conexão RAG estabelecida!")

## 5. Inferência e Demonstração

Comparação entre o Gemini Puro e o sistema completo BioByIA.

In [ ]:
# 4.1 CARREGAR ADAPTER
ADAPTER_HF = "vitateje/biobyai"
print(f"📥 Carregando Adapter: {ADAPTER_HF}")
model_ft, tokenizer_ft = FastLanguageModel.from_pretrained(
    model_name = ADAPTER_HF,
    max_seq_length = 2048,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model_ft)
print("✅ BioByIA pronto!")

In [ ]:
def inferencia_gemini_puro(pergunta):
    """Teste sem Fine-tuning e sem RAG (Gemini Direto)."""
    key = os.environ.get("GEMINI_API_KEY", "")
    if not key: raise ValueError("API Key do Gemini não encontrada!")
    
    genai.configure(api_key=key)
    model_base = genai.GenerativeModel("gemini-2.5-flash-lite")
    
    print("\n--- [TESTE 1] GEMINI FLASH-LITE (SEM RAG/FT) ---")
    try:
        response = model_base.generate_content(pergunta)
        print(f"Resposta: {response.text.strip()}")
    except Exception as e:
        print(f"❌ Erro na API do Gemini: {e}")

def inferencia_biobyia_rag(pergunta):
    """Teste com Fine-tuning + RAG (BioByIA)."""
    print("\n--- [TESTE 2] BIOBYIA (RAG + FINE-TUNING) ---")
    
    # 1. Busca Contexto
    results = query_rag_context(pergunta)
    context_text = "\n\n".join([f"[Fonte: {r['id']}]: {r['text']}" for r in results])
    
    # 2. Prompt
    prompt = f"""Responda como um assistente médico especialista.
Baseie sua resposta no contexto abaixo e cite os IDs das fontes.

CONTEXTO:
{context_text}

PERGUNTA: {pergunta}

RESPOSTA:"""
    
    inputs = tokenizer_ft(prompt, return_tensors="pt").to("cuda")
    outputs = model_ft.generate(**inputs, max_new_tokens=400)
    resposta = tokenizer_ft.decode(outputs[0], skip_special_tokens=True)
    
    if "RESPOSTA:" in resposta: resposta = resposta.split("RESPOSTA:")[-1].strip()
        
    print(f"Resposta: {resposta}")
    print("\nRastreabilidade (Fontes Utilizadas):")
    for r in results:
        print(f"- {r['id']} (Similaridade: {r['score']:.4f})")


In [ ]:
pergunta_final = "Explique o conceito de edição genética de forma simples."

inferencia_gemini_puro(pergunta_final)
inferencia_biobyia_rag(pergunta_final)